In [1]:
from glove import Glove
from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Activation, Dropout
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras_preprocessing.text import Tokenizer
from string import punctuation
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
glove = Glove()
glove = glove.load("/home/binoy/OneDrive/UH/COSC 6368 - AI/FinalProject/pos_glove_100d.model")

In [3]:
embed_dim = glove.word_vectors.shape[1]
vocab_len = len(glove.dictionary.keys()) + 1

In [4]:
emb_matrix = np.zeros((vocab_len, embed_dim))

In [5]:
embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_dim, trainable=False)
embedding_layer.build((None,))
embedding_layer.set_weights([emb_matrix])

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
def RNN(max_words, max_len):
    inputs = Input(name='inputs', shape=[max_len])
    layer = embedding_layer(inputs)
    layer = Bidirectional(LSTM(128, return_sequences=True))(layer)
    layer = LSTM(128)(layer)
    layer = Dense(64, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    print(model.summary())
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [7]:
data = pd.read_pickle('scaled_features.pkl')

In [8]:
pos = data.POS.values
pos = [tags.translate(str.maketrans('', '', punctuation)).strip() for tags in pos]

In [9]:
max_words = 300
tok = Tokenizer(num_words=max_words, filters='')
tok.fit_on_texts(pos)

In [10]:
max_len=250
sequences = tok.texts_to_sequences(pos)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

In [11]:
gender = data.Gender

In [12]:
le = LabelEncoder()
gender = le.fit_transform(gender)
gender = gender.reshape(-1, 1)

In [13]:
model = RNN(max_words, max_len)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 250)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 250, 100)          4200      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 250, 256)          234496    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
FC1 (Dense)                  (None, 64)                8256      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 65        
__________

In [ ]:
model.fit(sequences_matrix, gender, batch_size=128, epochs=10,
                      validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 2890 samples, validate on 322 samples
Epoch 1/10
2890/2890 [==============================] - 40s 14ms/step - loss: 0.6931 - acc: 0.5173 - val_loss: 0.6931 - val_acc: 0.5093
Epoch 2/10
2890/2890 [==============================] - 39s 14ms/step - loss: 0.6929 - acc: 0.5215 - val_loss: 0.6930 - val_acc: 0.5093
Epoch 3/10
2890/2890 [==============================] - 38s 13ms/step - loss: 0.6927 - acc: 0.5215 - val_loss: 0.6930 - val_acc: 0.5093
Epoch 4/10
2890/2890 [==============================] - 40s 14ms/step - loss: 0.6927 - acc: 0.5215 - val_loss: 0.6930 - val_acc: 0.5093
Epoch 5/10
2890/2890 [==============================] - 38s 13ms/step - loss: 0.6925 - acc: 0.5215 - val_loss: 0.6930 - val_acc: 0.5093
Epoch 6/10
2890/2890 [==============================] - 39s 13ms/step - loss: 0.6925 - acc: 0.5215 - val_loss: 0.6930 - val_acc: 0.5093
Epoch 7/10
1536/2890 [==============>...............] - ETA: 17s - loss: 0.6929 - acc: 0.

In [ ]:
model.evaluate(sequences_matrix, gender)